# Some tutorials
<a href="https://towardsdatascience.com/object-detection-explained-r-cnn-a6c813937a76"> region proposal using r-cnn</a>

<a href="https://learnopencv.com/selective-search-for-object-detection-cpp-python/"> region proposal using selective segmentation</a>

https://github.com/chingisooinar/Object-Detection_from-Scratch/blob/main/RCNN/RCNN.ipynb

<a href="https://towardsdatascience.com/step-by-step-r-cnn-implementation-from-scratch-in-python-e97101ccde55">RCNN-implemenation</a>

# Selective selection

In [1]:
cd

/home/dhruv


In [2]:
pwd

'/home/dhruv'

In [3]:
cd drone-defense-wall/data

/home/dhruv/drone-defense-wall/data


In [5]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline

In [5]:
#!/usr/bin/env python
'''
Usage:
./ssearch.py input_image (f|q)
f=fast, q=quality
Use "l" to display less rects, 'm' to display more rects, "q" to quit.
'''

import sys
import cv2


# If image path and f/q is not passed as command
# line arguments, quit and display help message
if len(sys.argv) < 3:
    print(__doc__)
    sys.exit(1)

# speed-up using multithreads
cv2.setUseOptimized(True);
cv2.setNumThreads(4);

# read image
im = cv2.imread("194.jpg")
# resize image
newHeight = 200
newWidth = int(im.shape[1]*200/im.shape[0])
im = cv2.resize(im, (newWidth, newHeight))    

# create Selective Search Segmentation Object using default parameters
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

# set input image on which we will run segmentation
ss.setBaseImage(im)

# Switch to fast but low recall Selective Search method
#if (sys.argv[2] == 'f'):
ss.switchToSelectiveSearchFast()

# Switch to high recall but slow Selective Search method
# elif (sys.argv[2] == 'q'):
#     ss.switchToSelectiveSearchQuality()
# # if argument is neither f nor q print help message
# else:
#     print(__doc__)
#     sys.exit(1)

# run selective search segmentation on input image
rects = ss.process()

# taking only the upper 2/3rd of the image
bboxes=[]
for x,y,w,h in rects:
    if y<100:
        bboxes.append([x,y,w,h])
bboxes=np.array(bboxes)
rects=bboxes
print('Total Number of Region Proposals: {}'.format(len(rects)))

# number of region proposals to show
numShowRects = 80
# increment to increase/decrease total number
# of reason proposals to be shown
increment = 10





while True:
    # create a copy of original image
    imOut = im.copy()

    # itereate over all the region proposals
    for i, rect in enumerate(rects):
        # draw rectangle for region proposal till numShowRects
        if (i < numShowRects):
            x, y, w, h = rect
            cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)


        else:
            break

    # show output
    cv2.imshow("Output", imOut)

    # record key press
    k = cv2.waitKey(0) & 0xFF

    # m is pressed
    if k == 109:
        # increase total number of rectangles to show by increment
        numShowRects += increment
    # l is pressed
    elif k == 108 and numShowRects > increment:
        # decrease total number of rectangles to show by increment
        numShowRects -= increment
    # q is pressed
    elif k == 113:
        break
# close image show window
cv2.destroyAllWindows()


Total Number of Region Proposals: 119


# Object Tracking
- using boosting and median tracking

In [6]:
cd

/home/dhruv


In [9]:
cd openCV-practice/Computer-Vision-with-Python/

/home/dhruv/openCV-practice/Computer-Vision-with-Python


In [7]:
def ask_for_tracker():
    print("Welcome! What Tracker API would you like to use?")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("")
    
    if choice == '0':
        tracker = cv2.legacy.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.legacy.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.legacy.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.legacy.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.legacy.TrackerMedianFlow_create()


    return tracker
 



In [8]:
tracker = ask_for_tracker()


Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 
3


In [9]:
import cv2
tracker = ask_for_tracker()
tracker_name = str(tracker).split()[0][1:]

# Read video
cap = cv2.VideoCapture("904.avi")

# Read first frame.
ret, frame = cap.read()


# Special function allows us to draw on the very first frame our desired ROI
## replace this roi by the coordinates from the resultant image from the DL model resultant
roi = cv2.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
ret = tracker.init(frame, roi)

while True:
    # Read a new frame
    ret, frame = cap.read()
    
    
    # Update tracker
    success, roi = tracker.update(frame)
    
    # roi variable is a tuple of 4 floats
    # We need each value and we need them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    # Draw Rectangle as Tracker moves
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
        print(f'{p1} and {p2}')
    else :
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    # Display tracker type on frame
    cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3);

    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()
cv2.destroyAllWindows()

Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 
3
(699, 425) and (787, 459)
(699, 428) and (787, 462)
(703, 430) and (790, 464)
(707, 432) and (792, 465)
(699, 435) and (787, 469)
(703, 438) and (789, 472)
(707, 441) and (792, 474)
(711, 443) and (794, 475)
(708, 442) and (796, 476)
(714, 445) and (799, 478)
(721, 445) and (804, 477)
(727, 444) and (809, 476)
(733, 444) and (813, 475)
(762, 453) and (823, 477)
(768, 451) and (829, 475)
(762, 453) and (823, 477)
(765, 454) and (825, 477)
(770, 456) and (821, 476)
(774, 456) and (824, 475)
(775, 456) and (826, 476)
(765, 456) and (816, 476)
(765, 460) and (816, 480)
(780, 460) and (831, 480)
(782, 462) and (832, 481)
(790, 462) and (841, 482)
(790, 460) and (841, 480)
(800, 464) and (851, 484)
(800, 466) and (851, 486)
(800, 468) and (851, 488)
(445, 297) and (823, 445)
(444, 298) and (822, 446)
(444, 300) and (822, 448)
(444, 301) and 

(672, 209) and (897, 297)
(674, 209) and (901, 298)
(687, 223) and (906, 308)
(689, 224) and (910, 310)
(687, 223) and (906, 308)
(689, 224) and (909, 310)
(691, 224) and (913, 311)
(691, 224) and (913, 311)
(690, 224) and (913, 311)
(688, 223) and (911, 310)
(687, 223) and (906, 308)
(685, 222) and (904, 308)
(683, 222) and (902, 307)
(682, 222) and (900, 307)
(698, 236) and (880, 307)
(696, 235) and (878, 306)
(693, 235) and (874, 306)
(680, 222) and (862, 293)
(676, 220) and (857, 291)
(672, 218) and (853, 289)
(644, 223) and (863, 308)
(641, 220) and (861, 306)
(644, 214) and (863, 299)
(644, 197) and (863, 282)
(644, 197) and (863, 282)
(639, 195) and (860, 281)
(633, 191) and (857, 278)
(623, 206) and (842, 291)
(623, 197) and (842, 282)
(618, 195) and (839, 281)
(612, 193) and (834, 280)
(607, 192) and (830, 279)
(603, 189) and (827, 276)
(599, 188) and (824, 276)
(596, 187) and (822, 275)
(593, 185) and (819, 273)
(591, 183) and (817, 271)
(590, 182) and (817, 271)
(589, 181) a

(473, 410) and (692, 496)
(475, 413) and (694, 499)
(477, 415) and (696, 501)
(480, 417) and (700, 503)
(486, 420) and (706, 506)
(493, 421) and (715, 508)
(500, 421) and (723, 508)
(508, 420) and (732, 507)
(514, 419) and (739, 507)
(520, 419) and (747, 508)
(524, 418) and (752, 507)
(528, 417) and (758, 507)
(532, 415) and (763, 505)
(537, 414) and (771, 505)
(580, 412) and (799, 497)
(586, 411) and (805, 496)
(590, 407) and (810, 493)
(593, 404) and (816, 491)
(597, 401) and (821, 488)
(627, 372) and (809, 443)
(629, 371) and (813, 443)
(632, 370) and (816, 442)
(633, 368) and (818, 440)
(635, 365) and (821, 438)
(644, 365) and (826, 436)
(623, 361) and (842, 446)
(625, 359) and (843, 444)
(628, 357) and (846, 442)
(631, 356) and (847, 440)
(623, 361) and (842, 446)
(624, 359) and (843, 444)
(626, 357) and (845, 443)
(628, 356) and (846, 441)
(630, 355) and (848, 440)
(644, 335) and (863, 420)
(648, 334) and (867, 420)
(651, 333) and (871, 419)
(654, 332) and (873, 418)
(655, 332) a

## To-do modifying the above code to take in corrdinates from the DL model resultant image

In [3]:
cd

/home/dhruv


In [10]:
type(roi)

tuple

In [1]:
# modified code
import cv2
tracker = ask_for_tracker()
tracker_name = str(tracker).split()[0][1:]

# Read video
cap = cv2.VideoCapture("DATA/cam0.mp4")

# Read first frame.
ret, frame = cap.read()


# Special function allows us to draw on the very first frame our desired ROI
## replace this roi by the coordinates from the resultant image from the DL model resultant
roi = (499.0, 305.0, 54.0, 57.0)
# Initialize tracker with first frame and bounding box
ret = tracker.init(frame, roi)

while True:
    # Read a new frame
    ret, frame = cap.read()
    
    
    # Update tracker
    success, roi = tracker.update(frame)
    
    # roi variable is a tuple of 4 floats
    # We need each value and we need them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    # Draw Rectangle as Tracker moves
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
        print(f'{p1} and {p2}')
    else :
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    # Display tracker type on frame
    cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3);

    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()
cv2.destroyAllWindows()

NameError: name 'ask_for_tracker' is not defined

In [11]:
roi

(499.0, 305.0, 54.0, 57.0)